In [47]:
import pandas as pd
import os
import math
import requests

# Cargar el archivo camaras.csv
meteo_df = pd.read_csv('estacionesmeteo.csv', delimiter=',')

# Función para limpiar las coordenadas y convertirlas a float
def limpiar_lat_long(valor):
    valor = valor.replace('°', '').replace('N', '').replace('W', '').strip()
    grados = float(valor)
    if 'W' in valor:
        grados = -grados
    return grados

# Aplicar la función a las columnas de latitud y longitud
meteo_df['Latitud'] = meteo_df['Latitud (Estimada)'].apply(limpiar_lat_long)
meteo_df['Longitud'] = meteo_df['Longitud (Estimada)'].apply(limpiar_lat_long)

# Eliminar las columnas originales de latitud y longitud
camaras_df = meteo_df.drop(columns=["Latitud (Estimada)", "Longitud (Estimada)"])

# Directorio donde están los otros CSV
directorio_datos = 'datos_calidad_aire/'

# Función para calcular la distancia en metros entre dos puntos
def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # Radio de la Tierra en metros
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    
    a = math.sin(delta_phi / 2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    return R * c  # Distancia en metros

# Función para obtener las coordenadas exactas de una estación usando Nominatim API
def obtener_coordenadas(nombre_estacion):
    url = f'https://nominatim.openstreetmap.org/search?q={nombre_estacion}&format=json&addressdetails=1&limit=1'
    response = requests.get(url)
    datos = response.json()
    
    if datos:
        lat = float(datos[0]['lat'])
        lon = float(datos[0]['lon'])
        return lat, lon
    else:
        return None, None

# Lista para almacenar los resultados
resultados = []

# Cargar y procesar cada uno de los archivos CSV en el directorio
for archivo in os.listdir(directorio_datos):
    if archivo.endswith('.csv'):
        archivo_path = os.path.join(directorio_datos, archivo)
        df = pd.read_csv(archivo_path, delimiter=';')
        
        # Extraer la segunda fila de las coordenadas latitud y longitud
        lat_aire = df.loc[0, 'geo_point_lat']
        lon_aire = df.loc[0, 'geo_point_lon']
        
        # Buscar la estación meteorológica más cercana
        distancia_minima = float('inf')
        estacion_cercana = None
        
        for index, row in camaras_df.iterrows():
            # Obtener las coordenadas exactas de la estación meteorológica
            lat_meteo, lon_meteo = obtener_coordenadas(row['Nombre'])
            
            # Si se obtienen coordenadas válidas, calcular la distancia
            if lat_meteo and lon_meteo:
                distancia = haversine(lat_aire, lon_aire, lat_meteo, lon_meteo)
                
                # Si es la estación más cercana, actualizar los valores
                if distancia < distancia_minima:
                    distancia_minima = distancia
                    estacion_cercana = row['Nombre']
        
        # Añadir el resultado de este archivo
        resultados.append({
            'archivo': archivo,
            'estacion_meteorologica': estacion_cercana,
            'distancia_m': distancia_minima,  # Distancia en metros
            'lat_aire': lat_aire,
            'lon_aire': lon_aire
        })

# Convertir los resultados a un DataFrame
resultados_df = pd.DataFrame(resultados)

# Mostrar los resultados
print(resultados_df)


ConnectionError: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Val%C3%A8ncia%20Carpesa%20Alqueries%20del%20Pelut&format=json&addressdetails=1&limit=1 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000002ADBACCC2B0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [41]:
meteo_df

,Nombre,Latitud (Estimada),Longitud (Estimada),Latitud,Longitud
0,València Carpesa Alqueries del Pelut,39.54° N,0.36° W,39.54,0.36
1,València el Saler Playa de la Garrofera,39.40° N,0.30° W,39.40,0.30
2,València Albors,39.49° N,0.37° W,39.49,0.37
3,València Altocúmulo,39.48° N,0.36° W,39.48,0.36
4,València Benimaclet,39.49° N,0.35° W,39.49,0.35
5,València Camins al Grau,39.46° N,0.33° W,39.46,0.33
6,València Colegio Diocesano San Juan Bosco,39.48° N,0.37° W,39.48,0.37
7,València Colegio Pureza de María-Grao,39.47° N,0.35° W,39.47,0.35
8,València el Pla del Real,39.47° N,0.34° W,39.47,0.34
9,València l'Albufera/Racó de l'Olla (Centre d'I...,39.35° N,0.33° W,39.35,0.33


In [14]:
# Agrupar los resultados por la columna 'archivo'
agrupado = resultados_df.groupby('archivo')

# Para ver un resumen por grupo, como la cantidad de elementos por archivo
resumen_por_archivo = agrupado.size()

# Si quieres obtener otras estadísticas, como el promedio de latitudes y longitudes
resumen_estadisticas = agrupado[['lat_cam', 'lon_cam']].mean()

# Mostrar el resumen por archivo
print(resumen_por_archivo)
print(resumen_estadisticas)


archivo
A01_AVFRANCIA_60m.csv          4
A02_BULEVARDSUD_60m.csv        4
A03_MOLISOL_60m.csv            3
A04_PISTASILLA_60m.csv         4
A07_VALENCIACENTRE_60m.csv    12
A08_DR_LLUCH_60m.csv           1
A10_OLIVERETA_60m.csv         14
A11_PATRAIX_60m.csv            4
W01_AVFRANCIA_10m.csv          4
dtype: int64
                              lat_cam   lon_cam
archivo                                        
A01_AVFRANCIA_60m.csv       39.459458 -0.342392
A02_BULEVARDSUD_60m.csv     39.449761 -0.394823
A03_MOLISOL_60m.csv         39.480746 -0.407678
A04_PISTASILLA_60m.csv      39.456930 -0.375758
A07_VALENCIACENTRE_60m.csv  39.470525 -0.376612
A08_DR_LLUCH_60m.csv        39.467500 -0.328357
A10_OLIVERETA_60m.csv       39.469634 -0.405591
A11_PATRAIX_60m.csv         39.458004 -0.402488
W01_AVFRANCIA_10m.csv       39.459458 -0.342392


In [25]:
import pandas as pd
import os
import numpy as np

# Radio de la Tierra en metros
R = 6371000  

# Función para calcular la distancia usando la fórmula de Haversine
def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    return R * c  # Devuelve la distancia en metros

# Cargar el archivo camaras.csv con el delimitador adecuado
camaras_df = pd.read_csv('camaras.csv', delimiter=';')

# Convertir geo_point_2d en dos columnas de latitud y longitud
camaras_df[['lat', 'lon']] = camaras_df['geo_point_2d'].str.split(',', expand=True).astype(float)

# Directorio donde están los otros CSV
directorio_datos = 'datos_calidad_aire/'

# Lista para almacenar los resultados
resultados = []

# Cargar y procesar cada uno de los archivos CSV en el directorio
for archivo in os.listdir(directorio_datos):
    if archivo.endswith('.csv'):
        # Cargar el archivo CSV
        archivo_path = os.path.join(directorio_datos, archivo)
        df = pd.read_csv(archivo_path, delimiter=';')
        
        # Extraer la segunda fila de las coordenadas latitud y longitud
        lat_aire = df.loc[0, 'geo_point_lat']
        lon_aire = df.loc[0, 'geo_point_lon']
        
        # Crear una nueva columna con la distancia en metros
        camaras_df['distancia'] = camaras_df.apply(
            lambda row: haversine(row['lat'], row['lon'], lat_aire, lon_aire), axis=1
        )
        
        # Ordenar por distancia y seleccionar las 3 más cercanas
        camaras_cercanas = camaras_df.nsmallest(3, 'distancia')
        
        # Añadir las cámaras más cercanas al resultado
        for index, row in camaras_cercanas.iterrows():
            resultados.append({
                'archivo': archivo,
                'OBJECTID': row['OBJECTID'],
                'CamaraID': row['Id. Càmera / Id. Cámara'],
                'lat_cam': row['lat'],
                'lon_cam': row['lon'],
                'lat_aire': lat_aire,
                'lon_aire': lon_aire,
                'distancia_metros': row['distancia']
            })

# Convertir los resultados a un DataFrame
resultados_df = pd.DataFrame(resultados)

# Mostrar los resultados
print(resultados_df)


                           archivo  OBJECTID  CamaraID    lat_cam   lon_cam  \
0            A01_AVFRANCIA_60m.csv    114500     10305  39.459421 -0.342445   
1            A01_AVFRANCIA_60m.csv    114720     10302  39.459447 -0.342328   
2            A01_AVFRANCIA_60m.csv    114501     10304  39.459478 -0.342451   
3          A02_BULEVARDSUD_60m.csv    114644     11505  39.449796 -0.394871   
4          A02_BULEVARDSUD_60m.csv    114645     11504  39.449716 -0.394855   
5          A02_BULEVARDSUD_60m.csv    114643     11502  39.449805 -0.394789   
6              A03_MOLISOL_60m.csv    114997     11403  39.480795 -0.407711   
7              A03_MOLISOL_60m.csv    114999     11404  39.480697 -0.407682   
8              A03_MOLISOL_60m.csv    114998     11402  39.480746 -0.407641   
9           A04_PISTASILLA_60m.csv    114827      2605  39.456974 -0.375790   
10          A04_PISTASILLA_60m.csv    114828      2602  39.456905 -0.375815   
11          A04_PISTASILLA_60m.csv    114825      26

In [26]:
# Seleccionar solo las columnas 'archivo', 'OBJECTID' y 'distancia_metros'
resultados_seleccionados = resultados_df[['archivo', 'CamaraID', 'distancia_metros']]

# Ordenar por 'distancia_metros' de menor a mayor
resultados_ordenados = resultados_seleccionados.sort_values(by='distancia_metros', ascending=True)

# Mostrar los resultados ordenados
print(resultados_ordenados)



                           archivo  CamaraID  distancia_metros
18      A07_VALENCIACENTRE_60m.csv     10511         21.850622
19      A07_VALENCIACENTRE_60m.csv     10510         30.972365
20      A07_VALENCIACENTRE_60m.csv     10512         32.842716
27           A10_OLIVERETA_60m.csv      6012         55.150776
28           A10_OLIVERETA_60m.csv      6011         74.889048
29           A10_OLIVERETA_60m.csv      6004         80.469873
21            A08_DR_LLUCH_60m.csv     14206         90.857402
6              A03_MOLISOL_60m.csv     11403        100.697863
7              A03_MOLISOL_60m.csv     11404        107.223281
8              A03_MOLISOL_60m.csv     11402        108.234047
9           A04_PISTASILLA_60m.csv      2605        141.358796
3          A02_BULEVARDSUD_60m.csv     11505        142.512813
10          A04_PISTASILLA_60m.csv      2602        146.864028
11          A04_PISTASILLA_60m.csv      2604        147.274821
4          A02_BULEVARDSUD_60m.csv     11504        148

In [21]:
resultados_ordenados.tail(15)

,archivo,OBJECTID,distancia_metros
17,A06_VIVERS_60m.csv,114576,343.347532
43,W04_VALENCIADT_10m.csv,114572,390.090884
44,W04_VALENCIADT_10m.csv,114571,391.097111
45,W05_VALENCIA_UPV_10m.csv,115005,431.585372
46,W05_VALENCIA_UPV_10m.csv,115007,439.740061
47,W05_VALENCIA_UPV_10m.csv,115006,442.107408
12,A05_POLITECNIC_60m.csv,114793,442.791728
24,A09_CABANYAL_60m.csv,114395,464.092217
25,A09_CABANYAL_60m.csv,114392,467.010859
26,A09_CABANYAL_60m.csv,114394,470.974459


In [27]:
# Filtrar las cámaras dentro del margen de 250 metros
dentro_250m = resultados_df[resultados_df['distancia_metros'] <= 350]

# Filtrar las cámaras fuera del margen de 250 metros
fuera_250m = resultados_df[resultados_df['distancia_metros'] > 350]

# Obtener los nombres de los archivos dentro y fuera del margen
archivos_dentro = dentro_250m['archivo'].unique()
archivos_fuera = fuera_250m['archivo'].unique()

# Mostrar los resultados
print("Archivos dentro del margen de 250 metros:")
print(archivos_dentro)

print("\nArchivos fuera del margen de 250 metros:")
print(archivos_fuera)


Archivos dentro del margen de 250 metros:
['A01_AVFRANCIA_60m.csv' 'A02_BULEVARDSUD_60m.csv' 'A03_MOLISOL_60m.csv'
 'A04_PISTASILLA_60m.csv' 'A06_VIVERS_60m.csv'
 'A07_VALENCIACENTRE_60m.csv' 'A08_DR_LLUCH_60m.csv'
 'A10_OLIVERETA_60m.csv' 'A11_PATRAIX_60m.csv' 'W01_AVFRANCIA_10m.csv'
 'W02_NAZARET_10m.csv' 'W04_VALENCIADT_10m.csv']

Archivos fuera del margen de 250 metros:
['A05_POLITECNIC_60m.csv' 'A09_CABANYAL_60m.csv'
 'W03_VALENCIAAEROPUERTO_10m.csv' 'W04_VALENCIADT_10m.csv'
 'W05_VALENCIA_UPV_10m.csv']


In [30]:
# Ordenar el DataFrame 'dentro_250m' por la columna 'CamaraID'
dentro_250m_ordenado = dentro_250m.sort_values(by='CamaraID')

# Mostrar el DataFrame ordenado
print(dentro_250m_ordenado)


                       archivo  OBJECTID  CamaraID    lat_cam   lon_cam  \
42      W04_VALENCIADT_10m.csv    114432       405  39.480556 -0.373524   
10      A04_PISTASILLA_60m.csv    114828      2602  39.456905 -0.375815   
11      A04_PISTASILLA_60m.csv    114825      2604  39.456955 -0.375700   
9       A04_PISTASILLA_60m.csv    114827      2605  39.456974 -0.375790   
15          A06_VIVERS_60m.csv    114572      3103  39.477013 -0.367617   
17          A06_VIVERS_60m.csv    114576      3105  39.476928 -0.367738   
16          A06_VIVERS_60m.csv    114571      3106  39.476969 -0.367688   
32         A11_PATRAIX_60m.csv    114509      4302  39.457962 -0.402452   
31         A11_PATRAIX_60m.csv    114508      4304  39.458048 -0.402523   
30         A11_PATRAIX_60m.csv    114839      4305  39.458044 -0.402449   
29       A10_OLIVERETA_60m.csv    114705      6004  39.469935 -0.405645   
28       A10_OLIVERETA_60m.csv    114648      6011  39.469820 -0.406376   
27       A10_OLIVERETA_60

In [31]:
# Guardar el DataFrame ordenado como un archivo CSV
dentro_250m_ordenado.to_csv('dentro_250m_ordenado.csv', index=False)

# Confirmación
print("El archivo ha sido guardado como 'dentro_250m_ordenado.csv'.")


El archivo ha sido guardado como 'dentro_250m_ordenado.csv'.


In [34]:
import pandas as pd

# Cargar el CSV en un DataFrame
df = pd.read_csv('dentro_250m_ordenado.csv')

# Extraer el nombre de la ubicación (antes del primer guion bajo) y asignarlo como columna 'ubicacion'
df['ubicacion'] = df['archivo'].str.split('_').str[0]

# Agrupar por la 'ubicacion' y obtener los CamaraID correspondientes
camara_ubicacion = df.groupby('ubicacion')['CamaraID'].apply(list).reset_index()

# Crear el diccionario para mapear cada ubicación con sus CamaraID
camera_fiware_mapping = {}

# Llenar el diccionario con las ubicaciones y los CamaraID asociados
for index, row in camara_ubicacion.iterrows():
    camera_fiware_mapping[row['ubicacion']] = row['CamaraID']

# Mostrar el diccionario de cámaras y ubicaciones
print(camera_fiware_mapping)


{'A01': [10302, 10304, 10305], 'A02': [11502, 11504, 11505], 'A03': [11402, 11403, 11404], 'A04': [2602, 2604, 2605], 'A06': [3103, 3105, 3106], 'A07': [10510, 10511, 10512], 'A08': [14203, 14204, 14206], 'A10': [6004, 6011, 6012], 'A11': [4302, 4304, 4305], 'W01': [10302, 10304, 10305], 'W02': [13902, 13903, 13904], 'W04': [405]}
